In [ ]:
from quantrocket.license import set_license

set_license("45036507-dfa7-11ee-ad56-5d738746c51e")

In [ ]:
from quantrocket.license import get_license_profile

license_profile = get_license_profile()
print(license_profile)

In [ ]:
from quantrocket.codeload import clone
clone("moonshot-intro",replace=True)

In [ ]:
from quantrocket.history import create_usstock_db
create_usstock_db("usstock-free-1d", bar_size="1 day", free=True)

In [ ]:
from quantrocket.history import collect_history
collect_history("usstock-free-1d")

In [ ]:
from quantrocket.master import get_securities
# by specifying sec_types='STK', we exclude ETFs, which are present in the sample data 
securities = get_securities(vendors="usstock", sec_types="STK")
securities.head()

In [ ]:
from quantrocket.master import create_universe
create_universe("usstock-free", sids=securities.index.tolist())

In [ ]:
filtered_securities = securities[securities.Delisted==False]
# also limit the number of columns for readability
filtered_securities = filtered_securities[["Symbol", "Exchange", "Name", "Delisted"]]
filtered_securities.head()

In [ ]:
create_universe("usstock-free-active", sids=filtered_securities.index.tolist())

In [17]:
#Dataset import
import pandas as pd
from quantrocket.history import download_history_file
sid_aapl = "FIBBG000B9XRY4"
download_history_file(
    "usstock-free-1d",
    start_date="2023-01-01",
    end_date="2023-12-31",
    sids=sid_aapl,
    fields=["Close"],
    filepath_or_buffer="aapl_prices.csv"
)

In [4]:
# read the aapl_prices.csv file
data = pd.read_csv("aapl_prices.csv")
df = pd.DataFrame(data)

In [5]:
df.head()

,Sid,Date,Close
0,FIBBG000B9XRY4,2023-01-03,124.2163
1,FIBBG000B9XRY4,2023-01-04,125.4975
2,FIBBG000B9XRY4,2023-01-05,124.1666
3,FIBBG000B9XRY4,2023-01-06,128.7352
4,FIBBG000B9XRY4,2023-01-09,129.2616


In [6]:
#Function to classify states as Bull, Bear or Flat
def classify_state(return_value):
    if return_value > 0.01:
        return 1
    elif return_value < -0.01:
        return -1
    else:
        return 0

In [7]:
df['returns']=df['Close'].pct_change()
df['state'] = df['returns'].apply(classify_state)

rows = [-1, 0, 1]
columns = [-1, 0, 1]

#Construct a state transition matrix
transition_distribution = pd.crosstab(index=df['state'].shift(), columns=df['state'], rownames=['Previous State'], colnames=['Current State'], margins=False)

#Before normalization
print(transition_distribution)

Current State   -1   0   1
Previous State            
-1.0             5  26   4
 0.0            23  94  41
 1.0             7  38  11


In [8]:
df['returns']

0           NaN
1      0.010314
2     -0.010605
3      0.036794
4      0.004089
         ...   
245   -0.005547
246   -0.002841
247    0.000518
248    0.002226
249   -0.005424
Name: returns, Length: 250, dtype: float64

In [9]:
df['state']

0      0
1      1
2     -1
3      1
4      0
      ..
245    0
246    0
247    0
248    0
249    0
Name: state, Length: 250, dtype: int64

In [11]:
for i in range(1,len(df)):
    s_d=df['state'][i-1]
    s_d_plus_1=df['state'][i]
    transition_distribution.loc[s_d,s_d_plus_1]+=1

transition_distribution=transition_distribution/transition_distribution.sum()

#After normalization
print(transition_distribution)

Current State         -1         0         1
Previous State                              
-1.0            0.142857  0.164557  0.071429
 0.0            0.657143  0.594937  0.732143
 1.0            0.200000  0.240506  0.196429


In [12]:
#Function to calculate value based on all transition combinations
def calculate_value_function(transition_distribution, num_days):
    value_function = [0] * num_days
    for d in range(num_days - 1, -1, -1):
        if d + 1 < len(transition_distribution):
            value_function[d] = max(
                transition_distribution.loc[-1.0, -1] * value_function[d + 1] + 1,
                transition_distribution.loc[-1.0, 0] * value_function[d + 1] - 1,
                transition_distribution.loc[-1.0, 1] * value_function[d + 1],
                transition_distribution.loc[0.0, -1] * value_function[d + 1] - 1,
                transition_distribution.loc[0.0, 0] * value_function[d + 1],
                transition_distribution.loc[0.0, 1] * value_function[d + 1] + 1,
                transition_distribution.loc[1.0, -1] * value_function[d + 1],
                transition_distribution.loc[1.0, 0] * value_function[d + 1] + 1,
                transition_distribution.loc[1.0, 1] * value_function[d + 1],
            )
        else:
            value_function[d] = 0    
    return value_function

In [13]:
value_function=calculate_value_function(transition_distribution,len(df))

In [14]:
#Function to calculate optimal buy points
def calculate_optimal_buy_points(returns,value_function):
    optimal_buy_points=[]
    s_d=0
    for i in range(1,len(returns)):
        s_d_plus_1=df['state'][i]
        if s_d==0 and s_d_plus_1==1:
            optimal_buy_points.append(i)
        s_d=s_d_plus_1
    return optimal_buy_points

In [15]:
print('Value function:\n', value_function)

Value function:
 [1.7321428571428572, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [16]:
optimal_buy_points = calculate_optimal_buy_points(df['returns'], value_function)

print('Optimal buy points:\n', optimal_buy_points)

Optimal buy points:
 [1, 6, 8, 12, 16, 21, 28, 30, 41, 50, 52, 59, 61, 69, 79, 85, 88, 94, 100, 103, 108, 110, 113, 117, 120, 123, 133, 142, 160, 164, 177, 187, 191, 207, 209, 212, 216, 218, 232, 234, 238]
